In [1]:
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import KFold

In [2]:
num_classes = 10
batch_size = 64
epochs = 10
input_shape = (28, 28, 1)

In [3]:
# preprocess the data
def load_dataset():
    # Loading data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    print(x_train.shape, y_train.shape)
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)


    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    return x_train, y_train, x_test, y_test

In [4]:
def prep_pixels(train ,test):
    # convert class vectors to binary class matrices
    x_train = train.astype('float32')
    x_test = test.astype('float32')
    x_train /= 255.0
    x_test /= 255.0

    return x_train, x_test

In [5]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform',input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # compile model
    opt_sgd = tf.keras.SGD(learning_rate=0.001, momentum=0.9,decay=1e-6, nesterov=False) 
    model.compile(
        optimizer=opt_sgd,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [6]:
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds,shuffle=True,random_state=1)
	for train_ix, test_ix in kfold.split(dataX):
		# define model 
		model = define_model()
		# select rows for train and test 
		x_train, y_train, x_test, y_test =dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]

		# fit model 
		history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), verbose=1, validation_split=0.2)

		# evaluate model
		_, acc = model.evaluate(x_test, y_test, verbose=0)
		print('%.3f' % (acc * 100.0))

		# stores scores 
		scores.append(acc)
		histories.append(history)

	return scores, histories

In [7]:
def run_test():
    # load dataset
    x_train, y_train, x_test, y_test = load_dataset()
    # prepare pixel data
    x_train, x_test = prep_pixels(x_train, x_test)
    # evaluate model
    scores, histories = evaluate_model(x_train, y_train)

In [8]:
run_test()

11493376/11490434 [==============================] - 8s 1us/step
(60000, 28, 28) (60000,)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


TypeError: prep_pixels() missing 2 required positional arguments: 'train' and 'test'